In [3]:
import data_utils as data_utils
import graph_builder as graph_builder
import os.path as osp
from utils import load_col_min_max, inverse_min_max_scaler

# 1. Set dataset name
dataset_name = 'Credit'  # Make sure this matches your naming!
is_regression = False    # Credit fraud is typically a classification task
credit_data = data_utils.load_dataset(dataset_name)
data_x, data_y, column_names = credit_data
col_min_max = load_col_min_max(dataset_name)

from utils import load_nam_checkpoint
ckpt_dir = r"./credit/fold_5/split_1/model_0/best_checkpoint"  # Adjust path as needed
nam, sess = load_nam_checkpoint(ckpt_dir)


Using checkpoint: ./credit/fold_5/split_1/model_0/best_checkpoint\model.ckpt-3880
Feature widths: [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Num input features: 30
INFO:tensorflow:Restoring parameters from ./credit/fold_5/split_1/model_0/best_checkpoint\model.ckpt-3880
✅ Restored NAM from checkpoint.


In [ ]:
from utils import get_test_predictions, compute_features, get_feature_predictions
import numpy as np

# Test set predictions
test_predictions = get_test_predictions(nam, data_x.astype(np.float32), sess)

# Feature-wise (shape function) predictions
unique_features = compute_features(dataset_name)
feature_predictions = get_feature_predictions(nam, dataset_name, sess)

# --- Evaluate RMSE or AUROC ---

is_regression = False
test_metric = graph_builder.calculate_metric(
    data_y, test_predictions, regression=is_regression)
metric_str = 'RMSE' if is_regression else 'AUROC'
print(f'{metric_str}: {test_metric:.4f}')


In [ ]:
from utils import prepare_feature_arrays, get_dataset_config, compute_mean_predictions

SINGLE_FEATURES_ORIGINAL, UNIQUE_FEATURES_ORIGINAL, UNIQUE_FEATURES = prepare_feature_arrays(
    data_x, column_names, col_min_max, inverse_min_max_scaler
)
COL_NAMES, FEATURE_LABEL_MAPPING, CATEGORICAL_NAMES = get_dataset_config(dataset_name, column_names)
avg_hist_data, MEAN_PRED, ALL_INDICES = compute_mean_predictions(
    data_x, column_names, UNIQUE_FEATURES, feature_predictions
)


In [ ]:
from utils import compute_mean_feature_importance, plot_mean_feature_importance
feature_names, importances = compute_mean_feature_importance(avg_hist_data, MEAN_PRED)
fig = plot_mean_feature_importance(feature_names, importances, dataset_name)


In [ ]:
from utils import plot_nam_contributions_with_density
plt = plot_nam_contributions_with_density(
    hist_data=avg_hist_data,
    unique_features=UNIQUE_FEATURES_ORIGINAL,
    single_features=SINGLE_FEATURES_ORIGINAL,
    categorical_names=CATEGORICAL_NAMES,
    col_mapping=COL_NAMES[dataset_name],
    feature_mapping=FEATURE_LABEL_MAPPING[dataset_name],
    mean_pred=MEAN_PRED,
    feature_to_use=column_names
)
